### Assignment 1: Discrete event modelling.

**Course:**     Simulation Modeling of Financial and Economic Systems <br>
**Student:**    Danis Alukaev <br>
**Email:**      d.alukaev@innopolis.university <br>
**Group:**      B19-DS-01 <br>

**Variant:**    Task 2 <br>

On average, cars with products arrive at some base each 30 minutes. The average unloading time of one machine is 1.5 hours. Unloading is carried out by two teams of loaders. On the territory of the base, no more than 4 cars can be in line waiting for unloading. Determine the performance of the queuing system.

### Prerequisites

In [1]:
import simpy
import random 

In [2]:
ARRIVAL_PERIOD = 30
UNLOADING_TIME = 90 
LOADERS_NUMBER = 2
QUEUE_LENGTH = 4

### Simulation design

In [3]:
def get_random_interval(mean):
    """
    The method returning the random integer from exponential distribution.
    Rate parameter is derived from the given average time for the action.
    :arg mean: the average time taking the action.
    :return: time interval from exponential distribution.
    """

    num_seconds = 60
    intensity = 1 / (mean * num_seconds)
    interval = int(random.expovariate(intensity) / num_seconds)
    return interval

In [4]:
class ProductBase:
    """
    Class representing the products' base involved in the task.
    It aggregates the loader resources and implements the unloading functionality.
    The time required by loaders is taken at random from exponential distribution.
    """

    def __init__(self, env, loader_number, unloading_time, queue_length):
        self.env = env
        self.loader_number = loader_number
        self.loaders = simpy.Resource(env, capacity=loader_number)
        self.unloading_time = unloading_time
        self.queue = []
        self.queue_length = queue_length
    
    def unload(self, car):
        interval = get_random_interval(self.unloading_time)
        yield self.env.timeout(interval)
        print(f"Car {car.id} was unloaded at {env.now:.2f}.")

In [5]:
class Car:
    """
    Class representing the truck carrying the goods from suppliers.
    Implements arriving mechanism: if the queue is full, the car is rejected 
    by product base. Otherwise, the product base accepts the truck and performs the 
    unloading procedure.  
    """

    def __init__(self, env, id):
        self.env = env
        self.id = id
    
    def arrive(self, product_base: ProductBase):
        if len(product_base.queue) > product_base.queue_length - 1:
            print(f"Car {self.id} was rejected by products base.")
            return
        with product_base.loaders.request() as request:
            product_base.queue.append(self.id)
            yield request 
            product_base.queue.remove(self.id)
            print(f"Car {self.id} enters the base at {self.env.now:.2f}.")
            yield env.process(product_base.unload(self))
            print(f"Car {self.id} leaves the base at {self.env.now:.2f}.")

In [6]:
def setup(env, arrival_period, unloading_time, loaders_number, queue_length):
    """
    The method used to configure all the classes implemented and start the 
    request flow.
    """

    product_base = ProductBase(env, loaders_number, unloading_time, queue_length)
    cars = []

    while True:
        interval = get_random_interval(arrival_period)
        yield env.timeout(interval)
        cars.append(Car(env, len(cars)+1))
        env.process(cars[-1].arrive(product_base))

In [7]:
env = simpy.Environment()
args = (env, ARRIVAL_PERIOD, UNLOADING_TIME, LOADERS_NUMBER, QUEUE_LENGTH)
env.process(setup(*args))
env.run(until=1000)

Car 1 enters the base at 21.00.
Car 2 enters the base at 50.00.
Car 2 was unloaded at 88.00.
Car 2 leaves the base at 88.00.
Car 3 enters the base at 88.00.
Car 1 was unloaded at 90.00.
Car 1 leaves the base at 90.00.
Car 4 enters the base at 126.00.
Car 3 was unloaded at 139.00.
Car 3 leaves the base at 139.00.
Car 5 enters the base at 139.00.
Car 5 was unloaded at 147.00.
Car 5 leaves the base at 147.00.
Car 6 enters the base at 147.00.
Car 6 was unloaded at 163.00.
Car 6 leaves the base at 163.00.
Car 7 enters the base at 163.00.
Car 7 was unloaded at 191.00.
Car 7 leaves the base at 191.00.
Car 8 enters the base at 191.00.
Car 8 was unloaded at 192.00.
Car 8 leaves the base at 192.00.
Car 9 enters the base at 192.00.
Car 4 was unloaded at 207.00.
Car 4 leaves the base at 207.00.
Car 10 enters the base at 228.00.
Car 10 was unloaded at 232.00.
Car 10 leaves the base at 232.00.
Car 11 enters the base at 233.00.
Car 9 was unloaded at 282.00.
Car 9 leaves the base at 282.00.
Car 12 ent